In [7]:
import torch 
from torch import nn

In [8]:

# Define the linear regression model with the pytorch `nn.Module` class

In [15]:
class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        #Make `b` and `w` be the parameters of the model
        #Wrap them with `nn.Parameter` 
        self.b = nn.Parameter(torch.randn(1,
                                        requires_grad=True,
                                        dtype=torch.float))
        self.w = nn.Parameter(torch.randn(1,
                                        requires_grad=True,
                                        dtype=torch.float))
    def forward(self, x):
        # Compute the outputs /predictions
        return self.b + self.w*x

In [22]:
#Example to construct a object
dummy = ManualLinearRegression()
dummy.parameters()
print(dummy.parameters())
print(list(dummy.parameters()))

<generator object Module.parameters at 0x7fb870742890>
[Parameter containing:
tensor([0.4617], requires_grad=True), Parameter containing:
tensor([0.2674], requires_grad=True)]


In [21]:
print(dummy.state_dict())

OrderedDict([('b', tensor([2.2082])), ('w', tensor([-0.6380]))])


In [16]:
#Data generation 
import numpy as np
import torch 


true_b = 1
true_w = 2 
N = 100

# set the random seed for numpy 
np.random.seed(43)

x= np.random.rand(N,1)
epsilon = (.1 * np.random.rand(N,1))

y = true_b + true_w *x + epsilon

# Generate training and validating sets 
idx = np.arange(N)
np.random.shuffle(idx)

# Use first 80 randowm indices for train
train_idx = idx[:int(N*.8)]
val_idx = idx[int(N*.8):]

# Generate train and validation sets
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]

# Data preparation 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Tranform the data from numpy array to torch tensor
x_train_tensor = torch.as_tensor(x_train).float().to(device)
y_train_tensor = torch.as_tensor(y_train).float().to(device)

In [17]:
# Sets learning rate
lr = 0.1 

# Step 0 : Initialize parameters 'b' and 'w' randomly
torch.manual_seed(42)

# Create a model and send it at once to the device
model = ManualLinearRegression().to(device) # 1)

# Define a SGD optimizer to update the parameters
#optimizer = torch.optim.SGD([b,w], lr=lr)
optimizer = torch.optim.SGD(model.parameters(),lr=lr)


#Define a MSE loss function 
loss_fn = nn.MSELoss(reduction="mean")  


# Define number of epochs 
n_epochs=1000

for epoch in range(n_epochs): 
    model.train()  #2)
    # Step 1: Computes the model's predicted output - forward pass
    # No more manula prediction 
    #yhat = b + w*x_train_tensor 
    yhat = model(x_train_tensor)  #3)
    
    # Step 2: Computes the loss
    # No more manual loss
    # error = (yhat - y_train_tensor)
    # loss = (error**2).mean()
    loss = loss_fn(yhat, y_train_tensor) #2
    
    # Step 3: Computes gradients for both 'b' and 'w' parameters
    loss.backward()
    
    # Step 4: Updates parameters using gradients and the learning rate
    # No more manual update
    # with torch.no_grade():
    #    b-=lr*b.grad
    #    w-=lr*w.grad
    optimizer.step()
    
    
    # Graident Zeroing
    # No more telling pytorch to let gradients go 
    #b.grad.zero()
    #w.grad.zero()
    optimizer.zero_grad()

#print(b,w)
#Inspect the parameters using its state_dict
print(model.state_dict())

OrderedDict([('b', tensor([1.0557])), ('w', tensor([1.9947]))])
